# Notes
We have 4D data:
- The hut name
- The date of access
- The date of booking interest
- Number of available beds

How can we visualize this best?
- [ ] Aggregate by week

Research questions:
- Which huts are bottlenecks?

In [89]:
from pathlib import Path

import numpy as np
import pandas as pd

In [2]:
DATA_DIR = Path("../daily/")

In [64]:
def concat_all_dailies():
    dfs = []
    for loc in DATA_DIR.glob("*.csv"):
        df = pd.read_csv(loc)
        df = df.rename(columns={df.columns[0]: "name"})

        if loc.name.startswith("2023"): # only do 2024 because its easier to work with week
            continue
            
        df["date"] = pd.to_datetime(loc.stem)
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

In [71]:
df = concat_all_dailies()
df.head()

,name,2024-06-01,2024-06-02,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,2024-06-09,...,2024-09-19,2024-09-20,2024-09-21,2024-09-22,2024-09-23,2024-09-24,2024-09-25,2024-09-26,2024-09-27,2024-09-28
0,Chalet Les Méandres (ex Tupilak),12,23,22,21,22,17,11,23,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gîte Michel Fagot,0,0,0,0,0,0,0,7,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Refuge du Fioux,3,16,18,18,10,10,17,0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Auberge du Truc,0,0,0,0,0,0,0,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,La Ferme à Piron,0,0,0,5,0,0,8,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Is there even availability?
- Given a hut $h$ and a day $d$
    - For all daily availability data $T$
        - Check the number of beds on day $t-1$ and $t$
        - Compute the difference
        - Only keep positive difference (bed become available again)
    - Sum over all these differences
    - This gives the total number of beds that have become available again

In [106]:
on_name_date = df.sort_values(['name', 'date'])
cols = [col for col in on_name_date.columns if col not in ['name', 'date']]
diffs = on_name_date[cols].diff()

mask = on_name_date.name != on_name_date.name.shift(1)
diffs[mask] = np.nan
diffs[diffs < 0] = 0 # ignore when beds become unavailable

diffs['name'] = on_name_date['name']
diffs.groupby("name").sum().to_csv("../tmp/test1.csv')

TODO not sure if this is correct